<a href="https://colab.research.google.com/github/11doris/jazz-maestro/blob/colab_word_embeddings/tune_similarity_doc2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall gensim -y

Found existing installation: gensim 3.6.0
Uninstalling gensim-3.6.0:
  Successfully uninstalled gensim-3.6.0


In [2]:
!pip install gensim

     |████████████████████████████████| 24.1 MB 2.7 kB/s 


In [3]:
import gensim
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pprint
import pandas as pd
import numpy as np
from collections import Counter
import plotly.express as px
from tqdm import tqdm 
from gensim.models.doc2vec import Doc2Vec
import pickle
import os

In [4]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [5]:
print(gensim.__version__)

4.1.2


# Initialization

## Download the Data

### Simplified chords, with duplicates

In [6]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1YvpsEo6OGObpFsxgfZ_nPtgs1u59ydJy' -O input_seq.txt


--2021-11-15 18:05:50--  https://docs.google.com/uc?export=download&id=1YvpsEo6OGObpFsxgfZ_nPtgs1u59ydJy
Resolving docs.google.com (docs.google.com)... 142.250.136.101, 142.250.136.102, 142.250.136.100, ...
Connecting to docs.google.com (docs.google.com)|142.250.136.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/1b00smlrlqod63jc9c4jr233o5lhomfv/1636999500000/14329102864480165501/*/1YvpsEo6OGObpFsxgfZ_nPtgs1u59ydJy?e=download [following]
--2021-11-15 18:05:51--  https://doc-10-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/1b00smlrlqod63jc9c4jr233o5lhomfv/1636999500000/14329102864480165501/*/1YvpsEo6OGObpFsxgfZ_nPtgs1u59ydJy?e=download
Resolving doc-10-4c-docs.googleusercontent.com (doc-10-4c-docs.googleusercontent.com)... 173.194.197.132, 2607:f8b0:4001:c1b::84
Connecting to doc-10-4c-docs.googleusercontent.com (doc-10

### Major Triads, Minor Triads and Dom7 chords:

In [7]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=13WKnD2PIZlJA0TjN9-IbFpD-M7eUCgrd' -O input_seq.txt

--2021-11-15 18:05:51--  https://docs.google.com/uc?export=download&id=13WKnD2PIZlJA0TjN9-IbFpD-M7eUCgrd
Resolving docs.google.com (docs.google.com)... 142.250.136.113, 142.250.136.101, 142.250.136.102, ...
Connecting to docs.google.com (docs.google.com)|142.250.136.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/0enru7bualhfa06dqee0f2vqseeu9e3o/1636999500000/14329102864480165501/*/13WKnD2PIZlJA0TjN9-IbFpD-M7eUCgrd?e=download [following]
--2021-11-15 18:05:51--  https://doc-08-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/0enru7bualhfa06dqee0f2vqseeu9e3o/1636999500000/14329102864480165501/*/13WKnD2PIZlJA0TjN9-IbFpD-M7eUCgrd?e=download
Resolving doc-08-4c-docs.googleusercontent.com (doc-08-4c-docs.googleusercontent.com)... 173.194.197.132, 2607:f8b0:4001:c1b::84
Connecting to doc-08-4c-docs.googleusercontent.com (doc-08

### Simplified chords, no repetitions

In [8]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=18aqk_RtyCssrtFljjGo9icwgTYXcmKck' -O input_seq.txt


--2021-11-15 18:05:52--  https://docs.google.com/uc?export=download&id=18aqk_RtyCssrtFljjGo9icwgTYXcmKck
Resolving docs.google.com (docs.google.com)... 142.250.125.113, 142.250.125.102, 142.250.125.138, ...
Connecting to docs.google.com (docs.google.com)|142.250.125.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/cesrvu694g4kcvm96h8d357ofjfc19fm/1636999500000/14329102864480165501/*/18aqk_RtyCssrtFljjGo9icwgTYXcmKck?e=download [following]
--2021-11-15 18:05:52--  https://doc-0k-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/cesrvu694g4kcvm96h8d357ofjfc19fm/1636999500000/14329102864480165501/*/18aqk_RtyCssrtFljjGo9icwgTYXcmKck?e=download
Resolving doc-0k-4c-docs.googleusercontent.com (doc-0k-4c-docs.googleusercontent.com)... 173.194.197.132, 2607:f8b0:4001:c1b::84
Connecting to doc-0k-4c-docs.googleusercontent.com (doc-0k

### Basic Chords, no repetitions

In [69]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1t12yHtoRp0jMn8nbaEeg3AX4LHgBMTZe' -O input_seq.txt


--2021-11-15 18:19:30--  https://docs.google.com/uc?export=download&id=1t12yHtoRp0jMn8nbaEeg3AX4LHgBMTZe
Resolving docs.google.com (docs.google.com)... 173.194.194.102, 173.194.194.113, 173.194.194.139, ...
Connecting to docs.google.com (docs.google.com)|173.194.194.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0g-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/6p7331q27d6ku7eamnh1rnateginqfqv/1637000325000/14329102864480165501/*/1t12yHtoRp0jMn8nbaEeg3AX4LHgBMTZe?e=download [following]
--2021-11-15 18:19:30--  https://doc-0g-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/6p7331q27d6ku7eamnh1rnateginqfqv/1637000325000/14329102864480165501/*/1t12yHtoRp0jMn8nbaEeg3AX4LHgBMTZe?e=download
Resolving doc-0g-4c-docs.googleusercontent.com (doc-0g-4c-docs.googleusercontent.com)... 173.194.197.132, 2607:f8b0:4001:c1b::84
Connecting to doc-0g-4c-docs.googleusercontent.com (doc-0g

### Meta Data

In [70]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Qs_SNXKBCeHOZO4VTwIPLR6OsPzEjk6q' -O input_meta.csv

--2021-11-15 18:19:33--  https://docs.google.com/uc?export=download&id=1Qs_SNXKBCeHOZO4VTwIPLR6OsPzEjk6q
Resolving docs.google.com (docs.google.com)... 142.250.125.102, 142.250.125.138, 142.250.125.139, ...
Connecting to docs.google.com (docs.google.com)|142.250.125.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/sa7treu0uim1lmhgc22kqec29360ggeo/1637000325000/14329102864480165501/*/1Qs_SNXKBCeHOZO4VTwIPLR6OsPzEjk6q?e=download [following]
--2021-11-15 18:19:34--  https://doc-14-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/sa7treu0uim1lmhgc22kqec29360ggeo/1637000325000/14329102864480165501/*/1Qs_SNXKBCeHOZO4VTwIPLR6OsPzEjk6q?e=download
Resolving doc-14-4c-docs.googleusercontent.com (doc-14-4c-docs.googleusercontent.com)... 173.194.197.132, 2607:f8b0:4001:c1b::84
Connecting to doc-14-4c-docs.googleusercontent.com (doc-14

### Read Data

In [71]:
path_to_file = '/content/input_seq.txt'

with open(path_to_file) as f: 
  lines = f.read().splitlines()

In [72]:
data = [line.split(' ') for line in lines]

### Meta Data

In [73]:
meta = pd.read_csv('input_meta.csv', sep='\t')
meta.columns

Index(['id', 'file_name', 'title', 'composer', 'year', 'year_truncated',
       'tonality', 'tune_key', 'tune_mode', 'structure', 'num_bars',
       'time_signature', 'cycle_fifths_order', 'style', 'musicbrainz_id',
       'wikidata_id', 'wikidata_allmusic', 'wiki_link', 'wikidata_description',
       'lyricist'],
      dtype='object')

In [74]:
titles = meta.loc[:, ['id', 'title', 'tune_mode']].drop_duplicates()
titles.set_index('id', inplace=True)

In [75]:
assert len(data) == len(titles)
print(len(data))

1812


In [76]:
titles_dict = titles.to_dict()
index_to_title = titles_dict['title']
index_to_title[1170]

'These Foolish Things'

In [77]:
title_to_index = {value: index for index, value in index_to_title.items()}
title_to_index['These Foolish Things']

1170

# Visualize Chord N-Grams



In [78]:
def ngrams(tokens, n=2, sep='-'):
    return [sep.join(ngram) for ngram in zip(*[tokens[i:] for i in range(n)])]

In [79]:
def list_to_ngrams(data, n=2, sep='-'):
  out = []
  for row in data:
    out.append(ngrams(row, n, sep))

  return out

In [80]:
data_ngrams = list_to_ngrams(data, 3)

In [81]:
def raw_chords_to_df(tunes):
  tunes_chords = [item for tune in tunes for item in tune]
  counts = Counter(tunes_chords)
  df = pd.DataFrame(counts.items(),
                    columns=['chord', 'count']).sort_values(by='count', ascending=False)

  return df

In [82]:
df_chords = raw_chords_to_df(data_ngrams)
df_chords.head(10)

,chord,count
14,Dm7-G7-C,3461
109,A7-Dm7-G7,1558
67,D7-G7-C,1164
34,Em7-A7-Dm7,1153
62,C-G7-C,859
155,A7-D7-G7,782
59,Am7-Dm7-G7,675
61,G7-C-G7,645
64,C-C7-F,639
6,Gm7-C7-F,625


In [83]:
df_chords = pd.DataFrame.from_dict(df_chords)
df_chords.sort_values(by=['count'], ascending=False, inplace=True)
df_chords_top = df_chords.query('count > 100')

fig = px.bar(df_chords_top, x='chord', y='count', log_y=True)
fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
fig.show()

# List Tunes for Test Data

In [84]:
contrafacts = {
               "26-2": "Confirmation",
               "52nd Street Theme": "I Got Rhythm",
               "Ablution": "All The Things You Are",
               "Anthropology": "I Got Rhythm",
               "Ballade": "As Long As I Live",
               "Bright Mississippi": "Sweet Georgia Brown",
               "C.T.A.": "I Got Rhythm",
               "Celia": "I Got Rhythm",
               "Cottontail": "I Got Rhythm",
               "Countdown": "Tune Up",
               "Dewey Square": "Oh, Lady Be Good",
               "Dexterity": "I Got Rhythm",
               "Dig": "Sweet Georgia Brown",
               "Donna Lee": "Indiana (Back Home Again In)",
               "Don't Be That Way": "I Got Rhythm",
               "Eternal Triangle": "I Got Rhythm",
               "Evidence": "Just You, Just Me",
               "Flintstones": "I Got Rhythm",
               "Four On Six": "Summertime",
               "Freight Train": "Blues For Alice",
               "Good Bait": "I Got Rhythm",
               "Hackensack": "Oh, Lady Be Good",
               "Half Nelson": "Lady Bird",
               "Hot House": "What Is This Thing Called Love",
               "Impressions": "So What",
               "In A Mellow Tone (In A Mellotone)": "Rose Room",
               "In Walked Bud": "Blue Skies",
               "Ko Ko": "Cherokee",
               "Lennie's Pennies": "Pennies From Heaven",   ## Lennie's Pennies is in minor and therefore transposed to Amin... not possible to recognize like that
               # "Let's Call This": "Honeysuckle Rose",
               "Little Rootie Tootie": "I Got Rhythm",
               "Little Willie Leaps": "All God's Chillun Got Rhythm",
               "Lullaby Of Birdland": "Love Me Or Leave Me",
               "Moose The Mooche": "I Got Rhythm",
               "My Little Suede Shoes": "Jeepers Creepers",
               "Oleo": "I Got Rhythm",
               "Ornithology": "How High The Moon",
               "Passport": "I Got Rhythm",
               "Quasimodo (Theme)": "Embraceable You",
               "Rhythm-a-ning": "I Got Rhythm",
               "Room 608": "I Got Rhythm",
               "Salt Peanuts": "I Got Rhythm",
               "Satellite": "How High The Moon",
               "Scrapple From The Apple": "Honeysuckle Rose", # A section
               "Scrapple From The Apple": "I Got Rhythm", # B section
               #"Segment": "I Got Rhythm",
               "Seven Come Eleven": "I Got Rhythm",
               "Shaw 'Nuff": "I Got Rhythm",
               "Theme, The": "I Got Rhythm",
               "Tour De Force": "Jeepers Creepers",
               "Wow": "You Can Depend On Me",
               "Yardbird Suite": "Rosetta",
               # following tunes are not from wikipedia:
               "Sweet Sue, Just You": "Honeysuckle Rose",  # A section
               # "All Of Me": "Pennies From Heaven", # bars 25-28 of All of Me are same as bars 17-20 of Pennies From Heaven, but different key!
               "Sweet Sue, Just You": "Bye Bye Blackbird", # Bridge same
               "These Foolish Things": "Blue Moon", # first 8 bars same
               
}

contrafacts_test = []
for reference, compare in contrafacts.items():
  contrafacts_test.append([title_to_index[reference], title_to_index[compare]])
  

In [85]:
contrafacts_test[:5]

[[0, 251], [3, 1527], [30, 63], [80, 1527], [104, 1372]]

# TF-IDF

## Build Model

In [86]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [87]:
tfidf_ngrams = [1, 2, 3, 4]

tfidf_model = {}
for n in tfidf_ngrams:

  data_ngrams = list_to_ngrams(data, n=n)
  data_tfidf = [" ".join(i) for i in data_ngrams]

  tfidf_vectorizer = TfidfVectorizer()
  tfidf_model[n] = tfidf_vectorizer.fit_transform(data_tfidf)

In [88]:
tfidf_model

{1: <1812x30 sparse matrix of type '<class 'numpy.float64'>'
 	with 15947 stored elements in Compressed Sparse Row format>,
 2: <1812x30 sparse matrix of type '<class 'numpy.float64'>'
 	with 15947 stored elements in Compressed Sparse Row format>,
 3: <1812x30 sparse matrix of type '<class 'numpy.float64'>'
 	with 15947 stored elements in Compressed Sparse Row format>,
 4: <1812x30 sparse matrix of type '<class 'numpy.float64'>'
 	with 15939 stored elements in Compressed Sparse Row format>}

## Test of the Contrafacts tunes

In [89]:
# create the empty dataframe for the results, with a column for each ngram value
cols = ['Tune1', 'Tune2']
for c in tfidf_model.keys():
  cols.append(f"cosval_ngram{c}")
df_eval_tfidf = pd.DataFrame(columns=cols)

# calculate a dataframe with the cos sim value for the test tunes
for tune1, tune2 in contrafacts_test:
  row = [index_to_title[tune1], 
         index_to_title[tune2]]
  for ngram_size, tfidf_m in tfidf_model.items():
    row.append(cosine_similarity(tfidf_m[tune1], tfidf_m[tune2])[0][0])
  
  df_eval_tfidf.loc[len(df_eval_tfidf)] = row

df_eval_tfidf

,Tune1,Tune2,cosval_ngram1,cosval_ngram2,cosval_ngram3,cosval_ngram4
0,26-2,Confirmation,0.568533,0.564252,0.574305,0.577075
1,52nd Street Theme,I Got Rhythm,0.848234,0.848234,0.841295,0.833057
2,Ablution,All The Things You Are,0.886735,0.877070,0.879810,0.878762
3,Anthropology,I Got Rhythm,0.758506,0.758506,0.766709,0.764410
4,Ballade,As Long As I Live,0.634559,0.646146,0.637085,0.628598
5,Bright Mississippi,Sweet Georgia Brown,0.889316,0.884625,0.874615,0.862008
6,C.T.A.,I Got Rhythm,0.624415,0.625980,0.635572,0.640375
7,Celia,I Got Rhythm,0.502398,0.486288,0.484689,0.471808
8,Cottontail,I Got Rhythm,0.913729,0.913729,0.909956,0.905746
9,Countdown,Tune Up,0.744414,0.725241,0.703373,0.691400


In [90]:
print(tfidf_model[2][1171].toarray())

[[0.22670044 0.         0.48745036 0.08954339 0.         0.0836956
  0.         0.         0.         0.         0.18985412 0.
  0.         0.23316521 0.         0.54771253 0.         0.
  0.         0.         0.         0.         0.21630323 0.
  0.10378447 0.         0.48053246 0.         0.09118211 0.09376861]]


In [91]:
m = tfidf_model[1]
tfidf_cos = cosine_similarity(m)

In [92]:
from operator import itemgetter

tfidf_dict = {}
for row_index, row in enumerate(tfidf_cos):
  indices, L_sorted = zip(*sorted(enumerate(row), key=itemgetter(1), reverse=True))
  
  # do not report self-similiarity, therefore remove the reference tune
  remove_index = indices.index(row_index)
  
  indices = list(indices)
  indices.pop(remove_index)
  L_sorted = list(L_sorted)
  L_sorted.pop(remove_index)

  tfidf_dict[row_index] = {}
  tfidf_dict[row_index]['tune_indices'] = indices[:10]
  tfidf_dict[row_index]['score'] = [round(num, 3) for num in L_sorted[:10]] 

tfidf_dict

{0: {'score': [0.764,
   0.757,
   0.749,
   0.738,
   0.734,
   0.726,
   0.726,
   0.718,
   0.697,
   0.696],
  'tune_indices': [425, 20, 939, 769, 192, 1215, 149, 262, 549, 356]},
 1: {'score': [0.8,
   0.798,
   0.772,
   0.749,
   0.726,
   0.718,
   0.717,
   0.703,
   0.697,
   0.693],
  'tune_indices': [338, 1005, 214, 1194, 852, 66, 716, 1294, 1235, 745]},
 2: {'score': [0.867,
   0.854,
   0.844,
   0.831,
   0.825,
   0.818,
   0.815,
   0.801,
   0.801,
   0.797],
  'tune_indices': [993, 282, 153, 1326, 1189, 913, 315, 912, 441, 50]},
 3: {'score': [0.982,
   0.968,
   0.962,
   0.962,
   0.959,
   0.957,
   0.944,
   0.942,
   0.942,
   0.928],
  'tune_indices': [516, 391, 651, 835, 767, 308, 92, 318, 457, 132]},
 4: {'score': [0.774,
   0.693,
   0.675,
   0.67,
   0.639,
   0.637,
   0.636,
   0.636,
   0.635,
   0.634],
  'tune_indices': [1416, 1141, 1627, 1793, 1031, 559, 107, 1574, 1313, 1478]},
 5: {'score': [0.881,
   0.864,
   0.863,
   0.848,
   0.831,
   0.829,


In [93]:
tfidf_file = 'tfidf_cossim.pickle'
with open(tfidf_file, 'wb') as f:
      # Pickle the 'data' dictionary using the highest protocol available.
      pickle.dump(tfidf_dict, f, pickle.HIGHEST_PROTOCOL)

from google.colab import files
files.download(tfidf_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Doc2Vec Embeddings

## Read and Tag Data

In [94]:
import smart_open

def read_corpus(fname, ngram_size, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            tokens = ngrams(line.split(), ngram_size, '-')
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

## Doc2Vec Hyperparameters

In [95]:
!rm -R models_doc2vec
!rm -R traindata_doc2vec

In [96]:
!mkdir models_doc2vec
!mkdir traindata_doc2vec

In [97]:
import multiprocessing

model_path = './models_doc2vec'
model_prefix = 'autos'
data_path = './traindata_doc2vec'

param_grid = {'d2v': {'variant': ['pv-dm', 
                                  'concat',
                                  'pv-dbow'
                                  ], 
                      'window': [
                                 2, 
                                 3, 
                                 #4, 
                                 #10
                                 ],
                      'sample': [0.1],
                      'ngrams': [
                                 1, 
                                 2, 
                                 #3, 
                                 #4
                                 ],
                     }
              }


common_kwargs = dict(
    vector_size=100, 
    epochs=20, 
    min_count=2,
    sample=0, 
    workers=multiprocessing.cpu_count(), 
    negative=10, 
    hs=0,
)

for algo, params in param_grid.items():
    for variant in params['variant']:
        dm = 0 if variant == 'pv-dbow' else 1
        concat = 1 if variant ==  'concat' else 0

        for ngram_size in params['ngrams']:
            
            # prepare data with the provided ngram size
            train_corpus = list(read_corpus(path_to_file, ngram_size=ngram_size))

            # evaluate different window sizes
            for window in params['window']:

                # evaluate the effect of different sample sizes
                for sample in params['sample']:

                    # calculate the model
                    if algo == 'd2v':
                        print(f"Model: {variant}, dm={dm}, dm_concat={concat}; ngrams={ngram_size}, window={window}, sample={sample}")
                        print(f"Corpus: {train_corpus[0]}")
                        model = gensim.models.doc2vec.Doc2Vec(**common_kwargs,
                                                              dm = dm,
                                                              dm_concat = concat,
                                                              )
                        model.build_vocab(train_corpus)
                        model.train(train_corpus, 
                                    total_examples=model.corpus_count, 
                                    epochs=model.epochs)

                    else:
                        model = gensim.models.FastText(data_ngrams, vector_size=size, window=window, sg=sg, epochs=50)

                    file_name = f"{model_path}/{model_prefix}_{algo}_{variant}_{ngram_size}_{window}_{sample}"
                    model.save(file_name + '.model')

                    file_name = f"{data_path}/{model_prefix}_{algo}_{variant}_{ngram_size}_{window}_{sample}"
                    with open(file_name + '.train', 'wb') as f:
                          # Pickle the 'data' dictionary using the highest protocol available.
                          pickle.dump(train_corpus, f, pickle.HIGHEST_PROTOCOL)

2021-11-15 18:19:44,298 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/m,d100,n10,w5,mc2,t2)', 'datetime': '2021-11-15T18:19:44.298839', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'created'}
2021-11-15 18:19:44,302 : INFO : collecting all words and their counts
2021-11-15 18:19:44,305 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2021-11-15 18:19:44,325 : INFO : collected 48 word types and 1812 unique tags from a corpus of 1812 examples and 70731 words
2021-11-15 18:19:44,327 : INFO : Creating a fresh vocabulary
2021-11-15 18:19:44,330 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=2 retains 48 unique words (100.0%% of original 48, drops 0)', 'datetime': '2021-11-15T18:19:44.330389', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18

Model: pv-dm, dm=1, dm_concat=0; ngrams=1, window=2, sample=0.1
Corpus: TaggedDocument(['C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'G#7', 'C#', 'E7', 'Am7', 'D7', 'Dm7', 'G7', 'C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C', 'Gm7', 'C7', 'Bm7', 'E7', 'A', 'C7', 'F', 'Bbm7', 'Eb7', 'G#', 'Dm7', 'G7', 'C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C'], [0])


2021-11-15 18:19:44,613 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-15 18:19:44,658 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-15 18:19:44,660 : INFO : EPOCH - 1 : training on 70731 raw words (72543 effective words) took 0.3s, 244090 effective words/s
2021-11-15 18:19:44,896 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-15 18:19:44,930 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-15 18:19:44,935 : INFO : EPOCH - 2 : training on 70731 raw words (72543 effective words) took 0.3s, 273669 effective words/s
2021-11-15 18:19:45,198 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-15 18:19:45,231 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-15 18:19:45,232 : INFO : EPOCH - 3 : training on 70731 raw words (72543 effective words) took 0.3s, 250068 effective words/s
2021-11-15 18:19:45,496 : INFO : worker thread finished

Model: pv-dm, dm=1, dm_concat=0; ngrams=1, window=3, sample=0.1
Corpus: TaggedDocument(['C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'G#7', 'C#', 'E7', 'Am7', 'D7', 'Dm7', 'G7', 'C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C', 'Gm7', 'C7', 'Bm7', 'E7', 'A', 'C7', 'F', 'Bbm7', 'Eb7', 'G#', 'Dm7', 'G7', 'C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C'], [0])


2021-11-15 18:19:50,672 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-15 18:19:50,703 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-15 18:19:50,705 : INFO : EPOCH - 1 : training on 70731 raw words (72543 effective words) took 0.3s, 237765 effective words/s
2021-11-15 18:19:50,973 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-15 18:19:51,000 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-15 18:19:51,001 : INFO : EPOCH - 2 : training on 70731 raw words (72543 effective words) took 0.3s, 252301 effective words/s
2021-11-15 18:19:51,274 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-15 18:19:51,298 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-15 18:19:51,300 : INFO : EPOCH - 3 : training on 70731 raw words (72543 effective words) took 0.3s, 250728 effective words/s
2021-11-15 18:19:51,566 : INFO : worker thread finished

...

2021-11-15 18:20:43,801 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-15 18:20:43,822 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-15 18:20:43,823 : INFO : EPOCH - 1 : training on 70731 raw words (72543 effective words) took 0.2s, 366263 effective words/s
2021-11-15 18:20:43,966 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-15 18:20:43,986 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-15 18:20:43,986 : INFO : EPOCH - 2 : training on 70731 raw words (72543 effective words) took 0.2s, 472661 effective words/s
2021-11-15 18:20:44,126 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-15 18:20:44,145 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-15 18:20:44,147 : INFO : EPOCH - 3 : training on 70731 raw words (72543 effective words) took 0.2s, 482030 effective words/s
2021-11-15 18:20:44,294 : INFO : worker thread finished

Model: pv-dbow, dm=0, dm_concat=0; ngrams=2, window=2, sample=0.1
Corpus: TaggedDocument(['C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-G#7', 'G#7-C#', 'C#-E7', 'E7-Am7', 'Am7-D7', 'D7-Dm7', 'Dm7-G7', 'G7-C', 'C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-C', 'C-Gm7', 'Gm7-C7', 'C7-Bm7', 'Bm7-E7', 'E7-A', 'A-C7', 'C7-F', 'F-Bbm7', 'Bbm7-Eb7', 'Eb7-G#', 'G#-Dm7', 'Dm7-G7', 'G7-C', 'C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-C'], [0])


2021-11-15 18:20:47,433 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-15 18:20:47,467 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-15 18:20:47,468 : INFO : EPOCH - 1 : training on 68919 raw words (70411 effective words) took 0.2s, 331251 effective words/s
2021-11-15 18:20:47,648 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-15 18:20:47,678 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-15 18:20:47,679 : INFO : EPOCH - 2 : training on 68919 raw words (70411 effective words) took 0.2s, 356593 effective words/s
2021-11-15 18:20:47,848 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-15 18:20:47,879 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-15 18:20:47,881 : INFO : EPOCH - 3 : training on 68919 raw words (70411 effective words) took 0.2s, 373741 effective words/s
2021-11-15 18:20:48,042 : INFO : worker thread finished

Model: pv-dbow, dm=0, dm_concat=0; ngrams=2, window=3, sample=0.1
Corpus: TaggedDocument(['C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-G#7', 'G#7-C#', 'C#-E7', 'E7-Am7', 'Am7-D7', 'D7-Dm7', 'Dm7-G7', 'G7-C', 'C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-C', 'C-Gm7', 'Gm7-C7', 'C7-Bm7', 'Bm7-E7', 'E7-A', 'A-C7', 'C7-F', 'F-Bbm7', 'Bbm7-Eb7', 'Eb7-G#', 'G#-Dm7', 'Dm7-G7', 'G7-C', 'C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-C'], [0])


2021-11-15 18:20:51,632 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-15 18:20:51,660 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-15 18:20:51,661 : INFO : EPOCH - 1 : training on 68919 raw words (70411 effective words) took 0.2s, 343564 effective words/s
2021-11-15 18:20:51,841 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-15 18:20:51,864 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-15 18:20:51,865 : INFO : EPOCH - 2 : training on 68919 raw words (70411 effective words) took 0.2s, 365376 effective words/s
2021-11-15 18:20:52,031 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-15 18:20:52,056 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-15 18:20:52,057 : INFO : EPOCH - 3 : training on 68919 raw words (70411 effective words) took 0.2s, 394071 effective words/s
2021-11-15 18:20:52,226 : INFO : worker thread finished

### Trial with Models according to the Original Paper 

In [98]:
import multiprocessing
from collections import OrderedDict

import gensim.models.doc2vec
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"

from gensim.models.doc2vec import Doc2Vec

train_corpus = list(read_corpus(path_to_file, ngram_size=1))

common_kwargs = dict(
    vector_size=100, 
    epochs=50, 
    min_count=2,
    sample=0, 
    workers=multiprocessing.cpu_count(), 
    negative=5, 
    hs=0,
)

simple_models = [
    # PV-DBOW plain
    Doc2Vec(dm=0, **common_kwargs),
    # PV-DM w/ default averaging; a higher starting alpha may improve CBOW/PV-DM modes
    Doc2Vec(dm=1, window=10, alpha=0.05, comment='alpha=0.05', **common_kwargs),
    # PV-DM w/ concatenation - big, slow, experimental mode
    # window=5 (both sides) approximates paper's apparent 10-word total window size
    Doc2Vec(dm=1, dm_concat=1, window=5, **common_kwargs),
]

for model in simple_models:
    model.build_vocab(train_corpus)
    model.train(train_corpus, 
                total_examples=model.corpus_count, 
                epochs=model.epochs)

    print("%s vocabulary scanned & state initialized" % model)

models_by_name = OrderedDict((str(model), model) for model in simple_models)

2021-11-15 18:20:55,558 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dbow,d100,n5,mc2,t2)', 'datetime': '2021-11-15T18:20:55.558747', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'created'}
2021-11-15 18:20:55,561 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/m,d100,n5,w10,mc2,t2)', 'datetime': '2021-11-15T18:20:55.561423', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'created'}
2021-11-15 18:20:55,564 : INFO : using concatenative 1100-dimensional layer1
2021-11-15 18:20:55,567 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/c,d100,n5,w5,mc2,t2)', 'datetime': '2021-11-15T18:20:55.567720', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event

Doc2Vec(dbow,d100,n5,mc2,t2) vocabulary scanned & state initialized


2021-11-15 18:21:03,459 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-15 18:21:03,486 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-15 18:21:03,488 : INFO : EPOCH - 1 : training on 70731 raw words (72543 effective words) took 0.3s, 255720 effective words/s
2021-11-15 18:21:03,750 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-15 18:21:03,774 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-15 18:21:03,776 : INFO : EPOCH - 2 : training on 70731 raw words (72543 effective words) took 0.3s, 262125 effective words/s
2021-11-15 18:21:04,022 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-15 18:21:04,053 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-15 18:21:04,055 : INFO : EPOCH - 3 : training on 70731 raw words (72543 effective words) took 0.3s, 272562 effective words/s
2021-11-15 18:21:04,302 : INFO : worker thread finished

Doc2Vec(dm/m,d100,n5,w10,mc2,t2) vocabulary scanned & state initialized


2021-11-15 18:21:17,599 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-15 18:21:17,640 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-15 18:21:17,641 : INFO : EPOCH - 1 : training on 70731 raw words (72543 effective words) took 0.3s, 229504 effective words/s
2021-11-15 18:21:17,913 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-15 18:21:17,958 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-15 18:21:17,959 : INFO : EPOCH - 2 : training on 70731 raw words (72543 effective words) took 0.3s, 235628 effective words/s
2021-11-15 18:21:18,209 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-15 18:21:18,247 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-15 18:21:18,250 : INFO : EPOCH - 3 : training on 70731 raw words (72543 effective words) took 0.3s, 258970 effective words/s
2021-11-15 18:21:18,505 : INFO : worker thread finished

Doc2Vec(dm/c,d100,n5,w5,mc2,t2) vocabulary scanned & state initialized


In [99]:
!pip install testfixtures

In [100]:
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
models_by_name['dbow+dmm'] = ConcatenatedDoc2Vec([simple_models[0], simple_models[1]])
models_by_name['dbow+dmc'] = ConcatenatedDoc2Vec([simple_models[0], simple_models[2]])

In [101]:
models_by_name

OrderedDict([('Doc2Vec(dbow,d100,n5,mc2,t2)',
              <gensim.models.doc2vec.Doc2Vec at 0x7f03a6cf0a50>),
             ('Doc2Vec(dm/m,d100,n5,w10,mc2,t2)',
              <gensim.models.doc2vec.Doc2Vec at 0x7f03a6cf0290>),
             ('Doc2Vec(dm/c,d100,n5,w5,mc2,t2)',
              <gensim.models.doc2vec.Doc2Vec at 0x7f03a6cedb90>),
             ('dbow+dmm',
              <gensim.test.test_doc2vec.ConcatenatedDoc2Vec at 0x7f03a6ce9110>),
             ('dbow+dmc',
              <gensim.test.test_doc2vec.ConcatenatedDoc2Vec at 0x7f03a6ce91d0>)])

In [102]:
from scipy import spatial

methods = ['Doc2Vec(dbow,d100,n5,mc2,t2)', 'Doc2Vec(dm/c,d100,n5,w5,mc2,t2)', 'dbow+dmm', 'dbow+dmc']

# create the empty dataframe for the results, with a column for each ngram value
cols = ['Tune1', 'Tune2', 'score_dbow', 'score_concat', 'score_dbow+dmm', 'score_dbow+dmc']

df_eval = pd.DataFrame(columns=cols)

# calculate a dataframe with the cos sim value for the test tunes
for tune1, tune2 in contrafacts_test:
    row = [index_to_title[tune1], index_to_title[tune2]]

    for m in methods:
        vec1 = models_by_name[m][tune1]
        vec2 = models_by_name[m][tune2]
      
        row.append(1 - spatial.distance.cosine(vec1, vec2))
  
    df_eval.loc[len(df_eval)] = row

df_eval

,Tune1,Tune2,score_dbow,score_concat,score_dbow+dmm,score_dbow+dmc
0,26-2,Confirmation,-0.091141,-0.037247,-0.210631,-0.024478
1,52nd Street Theme,I Got Rhythm,-0.041194,0.014654,0.091778,0.009437
2,Ablution,All The Things You Are,-0.023780,0.056288,0.063800,0.034725
3,Anthropology,I Got Rhythm,0.533002,0.489297,0.349535,0.512829
4,Ballade,As Long As I Live,0.432909,0.064900,0.548665,0.298529
5,Bright Mississippi,Sweet Georgia Brown,0.799263,0.654580,0.314742,0.737103
6,C.T.A.,I Got Rhythm,0.456349,0.086414,0.149662,0.298009
7,Celia,I Got Rhythm,0.217342,-0.123380,0.287106,0.071538
8,Cottontail,I Got Rhythm,0.838175,0.641179,0.782906,0.752063
9,Countdown,Tune Up,0.631912,0.761904,0.613325,0.680052


### End of Other Trial

In [103]:
from gensim.models import KeyedVectors

from os import listdir
from os.path import isfile, join

names = [f for f in listdir(model_path) if isfile(join(model_path, f))]
print(names)
models = {}

for name in names:
    file_name = f"{model_path}/{name}"
    print(file_name)
    models[name] = Doc2Vec.load(file_name)

2021-11-15 18:21:34,235 : INFO : loading Doc2Vec object from ./models_doc2vec/autos_d2v_concat_1_3_0.1.model
2021-11-15 18:21:34,241 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_concat_1_3_0.1.model.dv.* with mmap=None
2021-11-15 18:21:34,243 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_concat_1_3_0.1.model.wv.* with mmap=None
2021-11-15 18:21:34,246 : INFO : setting ignored attribute cum_table to None
2021-11-15 18:21:34,251 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_concat_1_3_0.1.model', 'datetime': '2021-11-15T18:21:34.251049', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}
2021-11-15 18:21:34,254 : INFO : loading Doc2Vec object from ./models_doc2vec/autos_d2v_pv-dm_2_2_0.1.model
2021-11-15 18:21:34,259 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_pv-dm_2_2_0.1.model.dv.* with mma

['autos_d2v_concat_1_3_0.1.model', 'autos_d2v_pv-dm_2_2_0.1.model', 'autos_d2v_pv-dbow_2_3_0.1.model', 'autos_d2v_pv-dm_2_3_0.1.model', 'autos_d2v_pv-dbow_2_2_0.1.model', 'autos_d2v_pv-dm_1_2_0.1.model', 'autos_d2v_pv-dm_1_3_0.1.model', 'autos_d2v_pv-dbow_1_2_0.1.model', 'autos_d2v_concat_2_2_0.1.model', 'autos_d2v_pv-dbow_1_3_0.1.model', 'autos_d2v_concat_2_3_0.1.model', 'autos_d2v_concat_1_2_0.1.model']
./models_doc2vec/autos_d2v_concat_1_3_0.1.model
./models_doc2vec/autos_d2v_pv-dm_2_2_0.1.model
./models_doc2vec/autos_d2v_pv-dbow_2_3_0.1.model
./models_doc2vec/autos_d2v_pv-dm_2_3_0.1.model
./models_doc2vec/autos_d2v_pv-dbow_2_2_0.1.model
./models_doc2vec/autos_d2v_pv-dm_1_2_0.1.model


2021-11-15 18:21:34,437 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_pv-dm_1_2_0.1.model.dv.* with mmap=None
2021-11-15 18:21:34,440 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_pv-dm_1_2_0.1.model.wv.* with mmap=None
2021-11-15 18:21:34,442 : INFO : setting ignored attribute cum_table to None
2021-11-15 18:21:34,447 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_pv-dm_1_2_0.1.model', 'datetime': '2021-11-15T18:21:34.447150', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}
2021-11-15 18:21:34,449 : INFO : loading Doc2Vec object from ./models_doc2vec/autos_d2v_pv-dm_1_3_0.1.model
2021-11-15 18:21:34,452 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_pv-dm_1_3_0.1.model.dv.* with mmap=None
2021-11-15 18:21:34,455 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_pv-dm_1_3_0.1.mod

./models_doc2vec/autos_d2v_pv-dm_1_3_0.1.model
./models_doc2vec/autos_d2v_pv-dbow_1_2_0.1.model
./models_doc2vec/autos_d2v_concat_2_2_0.1.model
./models_doc2vec/autos_d2v_pv-dbow_1_3_0.1.model
./models_doc2vec/autos_d2v_concat_2_3_0.1.model
./models_doc2vec/autos_d2v_concat_1_2_0.1.model


In [104]:
names = [f for f in listdir(data_path) if isfile(join(data_path, f))]
print(names)
data = {}

for name in names:
    file_name = f"{data_path}/{name}"
    print(file_name)
    with open(file_name, 'rb') as f:
        data[name] = pickle.load(f)

['autos_d2v_pv-dm_2_2_0.1.train', 'autos_d2v_concat_2_2_0.1.train', 'autos_d2v_concat_1_2_0.1.train', 'autos_d2v_pv-dbow_2_2_0.1.train', 'autos_d2v_pv-dm_1_2_0.1.train', 'autos_d2v_pv-dbow_1_3_0.1.train', 'autos_d2v_concat_2_3_0.1.train', 'autos_d2v_concat_1_3_0.1.train', 'autos_d2v_pv-dm_1_3_0.1.train', 'autos_d2v_pv-dm_2_3_0.1.train', 'autos_d2v_pv-dbow_2_3_0.1.train', 'autos_d2v_pv-dbow_1_2_0.1.train']
./traindata_doc2vec/autos_d2v_pv-dm_2_2_0.1.train
./traindata_doc2vec/autos_d2v_concat_2_2_0.1.train
./traindata_doc2vec/autos_d2v_concat_1_2_0.1.train
./traindata_doc2vec/autos_d2v_pv-dbow_2_2_0.1.train
./traindata_doc2vec/autos_d2v_pv-dm_1_2_0.1.train
./traindata_doc2vec/autos_d2v_pv-dbow_1_3_0.1.train
./traindata_doc2vec/autos_d2v_concat_2_3_0.1.train
./traindata_doc2vec/autos_d2v_concat_1_3_0.1.train
./traindata_doc2vec/autos_d2v_pv-dm_1_3_0.1.train
./traindata_doc2vec/autos_d2v_pv-dm_2_3_0.1.train
./traindata_doc2vec/autos_d2v_pv-dbow_2_3_0.1.train
./traindata_doc2vec/autos_d2v_p

In [105]:
for name, m in models.items():
  print(m)

Doc2Vec(dm/c,d100,n10,w5,mc2,t2)
Doc2Vec(dm/m,d100,n10,w5,mc2,t2)
Doc2Vec(dbow,d100,n10,mc2,t2)
Doc2Vec(dm/m,d100,n10,w5,mc2,t2)
Doc2Vec(dbow,d100,n10,mc2,t2)
Doc2Vec(dm/m,d100,n10,w5,mc2,t2)
Doc2Vec(dm/m,d100,n10,w5,mc2,t2)
Doc2Vec(dbow,d100,n10,mc2,t2)
Doc2Vec(dm/c,d100,n10,w5,mc2,t2)
Doc2Vec(dbow,d100,n10,mc2,t2)
Doc2Vec(dm/c,d100,n10,w5,mc2,t2)
Doc2Vec(dm/c,d100,n10,w5,mc2,t2)


In [106]:
def compare_doc_models(models, **kwargs):

    passes = 1

    df = pd.DataFrame()
    names = []
    scores = []
    runs = []

    for p in range(passes):
      print(f"\n*** Run {p}")
      for name, model in models.items():
        ranks = []
        second_ranks = []
        similarities = []

        print(name)

        # load the training corpus data
        train_name = os.path.splitext(name)[0] + '.train'
        train_corpus = data[train_name]
        print(train_corpus[:5])

        # use the model to infer the similarity for a tune
        for doc_id in range(len(train_corpus)):
            inferred_vector = model.infer_vector(train_corpus[doc_id].words)
            sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
            similarities.append(sims)
            rank = [docid for docid, sim in sims].index(doc_id)
            ranks.append(rank)

            second_ranks.append(sims[1])

        counter = Counter(ranks)
        print(counter)

        tunes_sim = counter[0]
        tunes_not_sim = len(train_corpus) - counter[0]
        score = 100 * tunes_not_sim/(tunes_sim+tunes_not_sim)

        print(f"\tTunes similar to themselves in first place: {tunes_sim}")
        print(f"\tTunes not similar in first place: {tunes_not_sim}")
        print(f"\t% of tunes not similar to themselves in first place: {score:.2f}%")

        names.append(name)
        scores.append(score)
        runs.append(p)
      
    df = pd.DataFrame({'name': names, 
                        'score': scores,
                        'pass': runs
                      })

    return df

In [107]:
df_score = compare_doc_models(models)


*** Run 0
autos_d2v_concat_1_3_0.1.model
[TaggedDocument(words=['C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'G#7', 'C#', 'E7', 'Am7', 'D7', 'Dm7', 'G7', 'C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C', 'Gm7', 'C7', 'Bm7', 'E7', 'A', 'C7', 'F', 'Bbm7', 'Eb7', 'G#', 'Dm7', 'G7', 'C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C'], tags=[0]), TaggedDocument(words=['Am7', 'Cm7', 'Eb', 'Em7', 'A7', 'Dm7', 'Bm7', 'Bbm7', 'Fm7', 'E7', 'Fm7', 'C#', 'Fm7', 'C#'], tags=[1]), TaggedDocument(words=['Am7', 'C#', 'Bm7', 'E7', 'Am7', 'C#', 'Bm7', 'E7', 'Cm7', 'F7', 'Bb', 'G#m7', 'C#7', 'F#m7', 'B7', 'E', 'E7', 'Am7', 'C#', 'Bm7', 'E7', 'Am7', 'C#', 'Bm7', 'E7', 'Cm7', 'F7', 'Bb', 'G#m7', 'C#7', 'F#m7', 'B7', 'Em7'], tags=[2]), TaggedDocument(words=['C', 'Am7', 'Dm7', 'G7', 'C', 'Am7', 'Dm7', 'G7', 'C', 'Am7', 'Dm7', 'G7', 'C', 'G7', 'C', 'Am7', 'Dm7', 'G7', 'C', 'Am7', 'Dm7', 'G7', 'C', 'Am7', 'Dm7', 'G7', 'C', 'G7',

In [108]:
pd.pivot_table(df_score, index = 'name', values = 'score', columns='pass')

pass,0
name,
autos_d2v_concat_1_2_0.1.model,18.763797
autos_d2v_concat_1_3_0.1.model,19.370861
autos_d2v_concat_2_2_0.1.model,94.977925
autos_d2v_concat_2_3_0.1.model,95.253863
autos_d2v_pv-dbow_1_2_0.1.model,4.249448
autos_d2v_pv-dbow_1_3_0.1.model,4.304636
autos_d2v_pv-dbow_2_2_0.1.model,3.697572
autos_d2v_pv-dbow_2_3_0.1.model,4.083885
autos_d2v_pv-dm_1_2_0.1.model,38.465784


In [109]:
def compare_doc_models2(models, **kwargs):

    passes = 1

    df = pd.DataFrame()
    topn = 5

    names = []
    scores = []
    runs = []
    for p in range(passes):
      print(f"\n*** Run {p}")
      for name, model in models.items():
        tune_sim = 0

        print(name)

        # load the training corpus data
        train_name = os.path.splitext(name)[0] + '.train'
        train_corpus = data[train_name]
        print(train_corpus[:5])

        # use the model to infer the similarity for a tune
        for doc_id in range(len(train_corpus)):
            inferred_vector = model.infer_vector(train_corpus[doc_id].words)
            sims = model.dv.most_similar([inferred_vector], topn=topn)
            if doc_id in [key for key, value in sims]:
              tune_sim += 1

        tunes_not_sim = len(train_corpus) - tune_sim
        score = 100 * tunes_not_sim/(tune_sim+tunes_not_sim)

        print(f"\tTunes similar to themselves in top {topn} place: {tune_sim}")
        print(f"\tTunes not similar in first place: {tunes_not_sim}")
        print(f"\t% of tunes not similar to themselves in first place: {score:.2f}%")

        names.append(name)
        scores.append(score)
        runs.append(p)
      
    df = pd.DataFrame({'name': names, 
                        'score': scores,
                        'pass': runs
                      })

    return df

In [110]:
df_score2 = compare_doc_models2(models)


*** Run 0
autos_d2v_concat_1_3_0.1.model
[TaggedDocument(words=['C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'G#7', 'C#', 'E7', 'Am7', 'D7', 'Dm7', 'G7', 'C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C', 'Gm7', 'C7', 'Bm7', 'E7', 'A', 'C7', 'F', 'Bbm7', 'Eb7', 'G#', 'Dm7', 'G7', 'C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C'], tags=[0]), TaggedDocument(words=['Am7', 'Cm7', 'Eb', 'Em7', 'A7', 'Dm7', 'Bm7', 'Bbm7', 'Fm7', 'E7', 'Fm7', 'C#', 'Fm7', 'C#'], tags=[1]), TaggedDocument(words=['Am7', 'C#', 'Bm7', 'E7', 'Am7', 'C#', 'Bm7', 'E7', 'Cm7', 'F7', 'Bb', 'G#m7', 'C#7', 'F#m7', 'B7', 'E', 'E7', 'Am7', 'C#', 'Bm7', 'E7', 'Am7', 'C#', 'Bm7', 'E7', 'Cm7', 'F7', 'Bb', 'G#m7', 'C#7', 'F#m7', 'B7', 'Em7'], tags=[2]), TaggedDocument(words=['C', 'Am7', 'Dm7', 'G7', 'C', 'Am7', 'Dm7', 'G7', 'C', 'Am7', 'Dm7', 'G7', 'C', 'G7', 'C', 'Am7', 'Dm7', 'G7', 'C', 'Am7', 'Dm7', 'G7', 'C', 'Am7', 'Dm7', 'G7', 'C', 'G7',

In [111]:
pd.pivot_table(df_score2, index = 'name', values = 'score', columns='pass')

pass,0
name,
autos_d2v_concat_1_2_0.1.model,7.836645
autos_d2v_concat_1_3_0.1.model,8.002208
autos_d2v_concat_2_2_0.1.model,92.825607
autos_d2v_concat_2_3_0.1.model,92.880795
autos_d2v_pv-dbow_1_2_0.1.model,0.275938
autos_d2v_pv-dbow_1_3_0.1.model,0.275938
autos_d2v_pv-dbow_2_2_0.1.model,0.165563
autos_d2v_pv-dbow_2_3_0.1.model,0.220751
autos_d2v_pv-dm_1_2_0.1.model,19.977925


In [112]:
import time
t = time.localtime()
timestamp = time.strftime('%Y%m%d_%H%M%S', t)
print(timestamp)

20211115_182635


In [113]:
score_filename = f'score_{timestamp}.csv'
df_score.to_csv(score_filename)

In [114]:
 !zip -r models.zip models_doc2vec/ 

updating: models_doc2vec/ (stored 0%)
updating: models_doc2vec/autos_d2v_concat_1_3_0.1.model (deflated 7%)
updating: models_doc2vec/autos_d2v_pv-dm_2_2_0.1.model (deflated 9%)
updating: models_doc2vec/autos_d2v_pv-dbow_2_3_0.1.model (deflated 9%)
updating: models_doc2vec/autos_d2v_pv-dm_2_3_0.1.model (deflated 9%)
updating: models_doc2vec/autos_d2v_pv-dbow_2_2_0.1.model (deflated 9%)
updating: models_doc2vec/autos_d2v_pv-dm_1_2_0.1.model (deflated 7%)
updating: models_doc2vec/autos_d2v_pv-dm_1_3_0.1.model (deflated 7%)
updating: models_doc2vec/autos_d2v_pv-dbow_1_2_0.1.model (deflated 7%)
updating: models_doc2vec/autos_d2v_concat_2_2_0.1.model (deflated 7%)
updating: models_doc2vec/autos_d2v_pv-dbow_1_3_0.1.model (deflated 7%)
updating: models_doc2vec/autos_d2v_concat_2_3_0.1.model (deflated 7%)
updating: models_doc2vec/autos_d2v_concat_1_2_0.1.model (deflated 7%)


In [115]:
from google.colab import files
files.download("models.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [116]:
files.download(score_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Assess the best Model

In [117]:
#best_model = 'autos_d2v_pv-dbow_3_4_0.1'
best_model = 'autos_d2v_concat_1_3_0.1'

In [118]:
model = models[f"{best_model}.model"]

#### Assess self-similarity

In [119]:
ranks = []
second_ranks = []
similarities = []

train_corpus = data[f"{best_model}.train"]

for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    similarities.append(sims)
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])

In [120]:
import collections

counter = collections.Counter(ranks)
print(counter)

tunes_sim = counter[0]
tunes_not_sim = len(train_corpus) - counter[0]

print()
print(f"Tunes similar to themselves in first place: {tunes_sim}")
print(f"Tunes not similar in first place: {tunes_not_sim}")
print(f"% of tunes not similar to themselves in first place: {100 * tunes_not_sim/(tunes_sim + tunes_not_sim):.2f}%")

Counter({0: 1461, 1: 115, 2: 45, 3: 31, 4: 22, 5: 15, 6: 10, 9: 10, 8: 9, 12: 8, 10: 7, 11: 6, 7: 5, 17: 5, 22: 4, 14: 3, 25: 3, 87: 3, 19: 2, 42: 2, 29: 2, 26: 2, 16: 2, 20: 2, 13: 2, 28: 2, 23: 2, 24: 2, 196: 1, 61: 1, 21: 1, 27: 1, 55: 1, 359: 1, 41: 1, 85: 1, 18: 1, 123: 1, 45: 1, 63: 1, 35: 1, 15: 1, 47: 1, 37: 1, 192: 1, 78: 1, 30: 1, 31: 1, 43: 1, 114: 1, 40: 1, 319: 1, 98: 1, 176: 1, 53: 1, 67: 1, 102: 1, 186: 1})

Tunes similar to themselves in first place: 1461
Tunes not similar in first place: 351
% of tunes not similar to themselves in first place: 19.37%


In [121]:
print('Tunes that are similar to other tunes before they are similar to themselves:')
for tune_index, value in enumerate(ranks):
  if value > 0:
    print(f"'{titles.iloc[tune_index, 0]}': {value}")

Tunes that are similar to other tunes before they are similar to themselves:
'After You': 6
'Alexander's Ragtime Band': 4
'All Alone': 1
'All God's Chillun Got Rhythm': 10
'All The Things You Are': 1
'All Through The Night': 2
'All Too Soon': 4
'Am I Blue': 5
'Among My Souvenirs': 3
'Anthropology': 2
'At Last': 9
'Bags and Trane': 1
'Bags' Groove': 5
'Barbados': 5
'Barbara': 6
'Basin Street Blues': 1
'Bein' Green': 4
'Bessie's Blues': 8
'Best Things In Life Are Free, The': 10
'Better Than Anything': 2
'Between The Devil And The Deep Blue Sea': 7
'Bill Bailey': 196
'Billie's Bounce': 5
'Billy Boy': 11
'Birth Of The Blues, The': 1
'Blackberry Winter': 1
'Blue Lou': 7
'Blue Monk': 19
'Blue Moon': 3
'Blue Room, The': 3
'Blue Skies': 1
'Blue Sphere': 14
'Bluehawk': 6
'Blues For Alice': 12
'Blues In The Closet': 2
'Bluesette': 42
'Booker's Waltz': 29
'Bourbon Street Parade': 61
'Brazil (Aquarela Do Brasil)': 1
'Bright Boy': 11
'Bright Mississippi': 2
'Bright Size Life': 1
'Broadway': 1
'But 

#### Determine Similarity of Tunes

In [122]:
assert(len(model.dv.vectors == len(list(title_to_index.keys()))))

In [123]:
tunes_eval_list = [
                   'Sweet Sue, Just You',
  'These Foolish Things', 
  'Blue Moon',
  'Hundred Years From Today, A',
  "All God's Chillun Got Rhythm",
  'I Got Rhythm',
  'Bye Bye Blackbird',
  'Old Fashioned Love',
  'Dinah',
  'Honeysuckle Rose',
  'Misty'
]

In [124]:
for tune in tunes_eval_list:
  sim = model.dv.most_similar(title_to_index[tune])

  print()
  print(f"'{tune}' is most similar to:")
  for index, value in sim:
    print(f'\t{value:.3f}: {index_to_title[index]}')



'Sweet Sue, Just You' is most similar to:
	0.896: Moten Swing
	0.891: Sweet Sue
	0.870: Rosalie
	0.780: This Year's Kisses
	0.779: Estate
	0.779: Three Little Words
	0.759: I've Got My Love To Keep Me Warm
	0.755: I Want To Be Happy
	0.747: Bossa Antigua
	0.742: Roses Of Picardy

'These Foolish Things' is most similar to:
	0.737: Bohemia After Dark
	0.708: Soul Eyes
	0.700: Witch Hunt
	0.687: S.O.S.
	0.669: Cottontail
	0.653: If I Should Lose You
	0.650: Impressions
	0.647: Catch Me
	0.646: On The Sunny Side Of The Street
	0.642: Waltz For Debby

'Blue Moon' is most similar to:
	0.720: There Will Never Be Another You
	0.703: Night Has A Thousand Eyes, The
	0.680: Making Whoopee
	0.667: I May Be Wrong
	0.652: My Heart Stood Still
	0.651: Night Dreamer
	0.641: Peggy O'Neil
	0.640: Avalon
	0.629: S'posin'
	0.621: Love For Sale

'Hundred Years From Today, A' is most similar to:
	0.706: 'S Wonderful
	0.677: Love Walked In
	0.664: Alabamy Bound
	0.664: Chicken
	0.648: Zing Went The Strings 

#### Evaluate Similarity for Jazz Contrafacts



In [125]:
# create the empty dataframe for the results, with a column for each ngram value
cols = ['Tune1', 'Tune2', 'doc2vec']
df_eval_doc2vec = pd.DataFrame(columns=cols)

# calculate a dataframe with the cos sim value for the test tunes
for tune1, tune2 in contrafacts_test:
  row = [index_to_title[tune1], 
         index_to_title[tune2]]
  row.append(model.dv.similarity(tune1, tune2))
  
  df_eval_doc2vec.loc[len(df_eval_doc2vec)] = row

df_eval_doc2vec

,Tune1,Tune2,doc2vec
0,26-2,Confirmation,0.140248
1,52nd Street Theme,I Got Rhythm,0.617661
2,Ablution,All The Things You Are,0.880569
3,Anthropology,I Got Rhythm,0.245398
4,Ballade,As Long As I Live,0.008885
5,Bright Mississippi,Sweet Georgia Brown,0.699102
6,C.T.A.,I Got Rhythm,0.011225
7,Celia,I Got Rhythm,-0.261150
8,Cottontail,I Got Rhythm,0.727402
9,Countdown,Tune Up,0.533785


##### Compare results from TF-IDF and doc2vec



In [126]:
df_eval_tfidf.merge(df_eval_doc2vec)

,Tune1,Tune2,cosval_ngram1,cosval_ngram2,cosval_ngram3,cosval_ngram4,doc2vec
0,26-2,Confirmation,0.568533,0.564252,0.574305,0.577075,0.140248
1,52nd Street Theme,I Got Rhythm,0.848234,0.848234,0.841295,0.833057,0.617661
2,Ablution,All The Things You Are,0.886735,0.877070,0.879810,0.878762,0.880569
3,Anthropology,I Got Rhythm,0.758506,0.758506,0.766709,0.764410,0.245398
4,Ballade,As Long As I Live,0.634559,0.646146,0.637085,0.628598,0.008885
5,Bright Mississippi,Sweet Georgia Brown,0.889316,0.884625,0.874615,0.862008,0.699102
6,C.T.A.,I Got Rhythm,0.624415,0.625980,0.635572,0.640375,0.011225
7,Celia,I Got Rhythm,0.502398,0.486288,0.484689,0.471808,-0.261150
8,Cottontail,I Got Rhythm,0.913729,0.913729,0.909956,0.905746,0.727402
9,Countdown,Tune Up,0.744414,0.725241,0.703373,0.691400,0.533785


##### Verbose Testing Output

In [127]:
similarity_found = 0

for tune1, tune2 in contrafacts.items():
  index1 = title_to_index[tune1]
  index2 = title_to_index[tune2]

  sim = model.dv.similarity(index1, index2)
  most_sim = model.dv.most_similar(index1, topn=5)
  
  # check whether the index2 was found among the topn similar tunes
  for i in most_sim:
    if index2 == i[0]:
      similarity_found += 1

  print()
  print(f"'{tune1}' to '{tune2}': {sim}")
  print(f"'{tune1}' is most similar to:")
  for index, value in most_sim:
    print(f'\t{value:.3f}: {index_to_title[index]}')


print()
num_tunes = len(contrafacts)
print(f"Number of tunes tested: {num_tunes}")
print(f"Number of similarities found: {similarity_found}")
print(f"Result: {100 * similarity_found / num_tunes}%")


'26-2' to 'Confirmation': 0.14024804532527924
'26-2' is most similar to:
	0.730: Whisper Not
	0.714: Bossa Antigua
	0.699: Drop Me Off In Harlem
	0.683: You Can Depend On Me
	0.681: Perhaps

'52nd Street Theme' to 'I Got Rhythm': 0.6176611185073853
'52nd Street Theme' is most similar to:
	0.869: I've Got My Fingers Crossed
	0.867: When The Moon Comes Over The Mountain
	0.854: Mack The Knife
	0.837: Why Do I Love You
	0.820: Soon

'Ablution' to 'All The Things You Are': 0.8805689811706543
'Ablution' is most similar to:
	0.881: All The Things You Are
	0.756: Everybody's Song But My Own
	0.712: Zingaro (Retrato Em Branco E Preto)
	0.712: Days Of Wine And Roses
	0.684: One Note Samba

'Anthropology' to 'I Got Rhythm': 0.24539807438850403
'Anthropology' is most similar to:
	0.975: Moose The Mooche
	0.968: Shaw 'Nuff
	0.964: Salt Peanuts
	0.963: No Moe
	0.957: Oleo

'Ballade' to 'As Long As I Live': 0.008885488845407963
'Ballade' is most similar to:
	0.813: Got A Match
	0.813: For Minors On

In [128]:
model.dv.similarity(0, 1)

0.45295945

#### Download Model

In [68]:
from google.colab import files
files.download(f"{model_path}/{best_model}.model")
files.download(f"{data_path}/{best_model}.train")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>